In [1]:
%%file deals.py

"""
Sue is a Rule One Investor.

She wants to check stock prices daily to see if any Rule One deals are available
for her.

For simplicity and robustness of examples, the `Ticker` class in these examples
is replaced with MockTicker.

When Sue uses this program, she always omits the part `Ticker=MockTicker`.


One day, Sue wants to check the price of a single stock.

>>> company = Company(symbol="tsm", sticker_price=11)
>>> get_deal(company, Ticker=MockTicker)
Deal(symbol='tsm', sticker_price=11, price=98, percent_of_sticker=891)


On another day, Sue has several companies she wants to check the price of.

To the same method `get_deal`, she can pass a list of companies the same way
she passed a single company.

>>> companies = [
...  Company(symbol="msft", sticker_price=118),
...  Company(symbol="tsm", sticker_price=11)
... ]
>>> deals = get_deal(companies, Ticker=MockTicker)
>>> deals[0]
Deal(symbol='msft', sticker_price=118, price=352.5, percent_of_sticker=299)
>>> deals[1]
Deal(symbol='tsm', sticker_price=11, price=98, percent_of_sticker=891)


Sue wants the results to be sorted in the order of the best deal available.

Best deal means the largest Margin of Safety (MOS).

If companies are not sorted in the order of the best deal, the results *should*
be.

Note the order of input companies is reversed with respect to the previous example.

>>> companies = [
...  Company(symbol="tsm", sticker_price=11),
...  Company(symbol="msft", sticker_price=118)
... ]
>>> deals = get_deal(companies, Ticker=MockTicker)
>>> deals[0]
Deal(symbol='msft', sticker_price=118, price=352.5, percent_of_sticker=299)


Sue says the interface is too complicated for her, and she would love to
be able to initialize Deal instances directly from Company instances.

>>> companies = [
...  Company(symbol="tsm", sticker_price=11),
...  Company(symbol="msft", sticker_price=118)
... ]
>>> deals = Deal.from_(companies, Ticker=MockTicker)
>>> deals[0]
Deal(symbol='msft', sticker_price=118, price=352.5, percent_of_sticker=299)
"""

from typing import NamedTuple
from yfinance import Ticker
from requests import get
from functools import lru_cache


class Company(NamedTuple):
    symbol: str
    sticker_price: float


class Deal(NamedTuple):

    symbol: str
    sticker_price: float
    price: float
    percent_of_sticker: int

    """
    Sue wants to print an instance of this class and be able to copy-paste
    it into another place, or save it for later pasting.

    >>> Deal(symbol='msft', sticker_price=1, price=1)
    Deal(symbol='msft', sticker_price=1, price=1)


    Sue wants to be able to compare instances of `Deal` to check for equality,
    in order to remove duplicates from lists if any are found.

    She does not remember, *why* exactly she wanted this feature. Still, she
    insists it should be available.

    >>> deal_1 = Deal(symbol='msft', sticker_price=1, price=1)
    >>> deal_2 = Deal(symbol='msft', sticker_price=1, price=1)
    >>> deal_1 == deal_2
    True
    """

    def __eq__(self, other):
        if repr(other) != repr(self):
            return False
        return True

    def __lt__(self, other):
        return self.percent_of_sticker < other.percent_of_sticker

    @staticmethod
    def from_(company, Ticker=Ticker):
        deal = get_deal(company, Ticker=Ticker)
        return deal


class MockTicker:
    def __init__(self, symbol):
        price = 352.5 if symbol == "msft" else 98

        self.info = {
            "currentPrice": price
        }


def get_price(symbol, Ticker=Ticker):

    """
    >>> get_price("msft", Ticker=MockTicker)
    352.5
    >>> get_price([])
    {}
    >>> get_price(["msft"], Ticker=MockTicker)
    {'msft': 352.5}
    """

    if type(symbol) == str:
        price = Ticker(symbol).info["currentPrice"]
        return price

    prices = {
        the_symbol: get_price(the_symbol, Ticker=Ticker)
        for the_symbol in symbol
    }
    return prices

def get_percent_of_sticker(price, sticker_price):
    return int(100 * round(price/sticker_price, 2))

def get_deal(company, Ticker=Ticker):

    """
    >>> company = Company(symbol="msft", sticker_price=118)
    >>> deal = get_deal(company, Ticker=MockTicker)
    >>> deal
    Deal(symbol='msft', sticker_price=118, price=352.5, percent_of_sticker=299)
    """

    if isinstance(company, list):
        return get_deals(company, Ticker=Ticker)

    price = get_price(company.symbol, Ticker=Ticker)
    return Deal(
        symbol=company.symbol,
        sticker_price=company.sticker_price,
        price=price,
        percent_of_sticker=get_percent_of_sticker(price, company.sticker_price)
    )

def get_deals(companies, Ticker=Ticker):

    """
    >>> companies = [
    ...   Company("msft", 118),
    ...   Company("tsm", 22)
    ... ]
    >>> deals = get_deals(companies, Ticker=MockTicker)
    >>> deals[0]
    Deal(symbol='msft', sticker_price=118, price=352.5, percent_of_sticker=299)
    """

    sticker_prices = {
        company.symbol: company.sticker_price
        for company in companies
    }

    deals = []
    for symbol in sticker_prices.keys():
        price = get_price(symbol, Ticker=Ticker)
        sticker_price=sticker_prices[symbol]
        deals.append(
            Deal(symbol=symbol,
                 sticker_price=sticker_price,
                 price=price,
                 percent_of_sticker=get_percent_of_sticker(price, sticker_price)
            )
        )

    return sorted(deals)

def mock_get_sticker_price(symbol):
    class MockResponse:
        def json():
            return {"sticker_price": {"value": 22}}

    return MockResponse

@lru_cache
def get_sticker(symbol, api_host="143.42.16.225:8080", get=get):
    """
    >>> round(get_sticker("tsm", get=mock_get_sticker_price))
    22
    """
    url = f"http://{api_host}/search/{symbol}"
    result = get(url).json()["sticker_price"]["value"]
    return result

def round_sticker(sticker):

    """
    Prices above $2 should be rounded to the nearest integer.
    >>> round_sticker(22.01)
    22

    Prices below $2 should be rounded to full cents.
    >>> round_sticker(0.512)
    0.51
    """
    return round(sticker) if sticker >= 2 else round(sticker, 2)

def is_valid_sticker(result):
    return result is not None and result >= 0

def get_stickers(items):
    stickers = {}
    for key in items.keys():
        result = get_sticker(key)
        if is_valid_sticker(result):
            stickers[key] = round_sticker(result)
    return stickers

def with_preview(value):
    print(value)
    return value

Writing deals.py


In [2]:
!pytest --doctest-modules deals.py

============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.2.2, pluggy-1.0.0
rootdir: /content
plugins: anyio-3.6.2
collected 6 items                                                              

deals.py ......                                                          [100%]

============================== 6 passed in 0.77s ===============================


In [3]:
# symbol: sticker_price
watch_list_items = {
    'TSM': 22,
    'ON': 24,
    'TSLA': 51,
    'MSFT': 118,
    'APH': 23,
    'LSCC': 0.51,
    'ACN': 114,
    'GOOG': 94,
    'ADBE': 115,
    'ADI': 37,
    'ANSS': 72,
    'AMAT': 72,
    'MELI': 2500,
    'MCHP': 44,
    'TSCO': 89,
    'SNPS': 50,
    'CPRT': 145,
    'UMC': 1.0,
    'ARCB': 75,
    'MBUU': 100,
    'PERI': 58,
    'NVR': 721,
    'WSO': 52,
    'WEEV': None,
    'UNH': 349,
    'HOG': 17,
    'ULTA': 354,
    'TW': 84,
    'CMG': 201,
    'ADSK': 240,
    'DE': 435,
    'LSTR': 145
}

In [4]:
from deals import with_preview, get_stickers

stickers = with_preview(get_stickers(watch_list_items))

{'TSM': 22, 'ON': 24, 'TSLA': 50, 'MSFT': 118, 'APH': 23, 'LSCC': 0.51, 'ACN': 120, 'GOOG': 92, 'ADBE': 115, 'ADI': 37, 'ANSS': 57, 'AMAT': 72, 'MCHP': 44, 'TSCO': 89, 'SNPS': 49, 'CPRT': 146, 'UMC': 0.99, 'ARCB': 75, 'MBUU': 95, 'PERI': 56, 'NVR': 7696, 'WSO': 52, 'HOG': 18, 'ULTA': 336, 'CMG': 202, 'DE': 433, 'LSTR': 145}


In [5]:
watch_list_items.update(stickers)
watch_list_items

{'TSM': 22,
 'ON': 24,
 'TSLA': 50,
 'MSFT': 118,
 'APH': 23,
 'LSCC': 0.51,
 'ACN': 120,
 'GOOG': 92,
 'ADBE': 115,
 'ADI': 37,
 'ANSS': 57,
 'AMAT': 72,
 'MELI': 2500,
 'MCHP': 44,
 'TSCO': 89,
 'SNPS': 49,
 'CPRT': 146,
 'UMC': 0.99,
 'ARCB': 75,
 'MBUU': 95,
 'PERI': 56,
 'NVR': 7696,
 'WSO': 52,
 'WEEV': None,
 'UNH': 349,
 'HOG': 18,
 'ULTA': 336,
 'TW': 84,
 'CMG': 202,
 'ADSK': 240,
 'DE': 433,
 'LSTR': 145}

In [6]:
# Import classes, reload if needed.
from importlib import reload; import deals; reload(deals)
from deals import Company, Deal

#
companies = [
    Company(symbol=symbol, sticker_price=watch_list_items[symbol])
    for symbol in watch_list_items.keys()
    if watch_list_items[symbol] is not None
]

deals = Deal.from_(companies)
deals

[Deal(symbol='MELI', sticker_price=2500, price=1214.191, percent_of_sticker=49),
 Deal(symbol='PERI', sticker_price=56, price=33.23, percent_of_sticker=59),
 Deal(symbol='CPRT', sticker_price=146, price=88.99, percent_of_sticker=61),
 Deal(symbol='MBUU', sticker_price=95, price=58.31, percent_of_sticker=61),
 Deal(symbol='NVR', sticker_price=7696, price=6013.93, percent_of_sticker=78),
 Deal(symbol='TW', sticker_price=84, price=69.34, percent_of_sticker=83),
 Deal(symbol='ADSK', sticker_price=240, price=208.105, percent_of_sticker=87),
 Deal(symbol='DE', sticker_price=433, price=417.45, percent_of_sticker=96),
 Deal(symbol='ARCB', sticker_price=75, price=87.52, percent_of_sticker=117),
 Deal(symbol='LSTR', sticker_price=145, price=185.42, percent_of_sticker=128),
 Deal(symbol='GOOG', sticker_price=92, price=121.345, percent_of_sticker=132),
 Deal(symbol='ULTA', sticker_price=336, price=451.03, percent_of_sticker=134),
 Deal(symbol='UNH', sticker_price=349, price=477.225, percent_of_sti